# <a>Projeto 1 - Análise Exploratória de Dados</a>

## <a> Motivação </a>

Todo trabalho de Ciência de Dados passa por essa importante etapa. Conhecer bem os dados, saber as distribuições, medidas estatísticas principais, visualizar correlações e evoluções é essencial para gerar insights e solucionar problemas.

Essa etapa é bem *híbrida* no sentido em que pode ser utilizada sem um objetivo completamente definido, numa busca por informações e insights. De outro lado, num projeto com objetivo de negócio bem definido (fase 1 do CRISP-DM), ela é bem focalizada nas perguntas de negócio que queremos responder

## <a> Objeto de Estudo </a>

Vamos trabalhar com informações de microdados no ENEM realizado em 2019. De acordo com o site do [INEP](https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem), o ENEM tem o objetivo de avaliar o desempenho escolar dos estudantes ao término da educação básica. As notas do Enem podem ser usadas para acesso ao Sistema de Seleção Unificada (Sisu) e ao Programa Universidade para Todos (ProUni). Elas também são aceitas em mais de 50 instituições de educação superior portuguesas. Além disso, os participantes do Enem podem pleitear financiamento estudantil em programas do governo, como o Fundo de Financiamento Estudantil (Fies). Os resultados do Enem possibilitam, ainda, o desenvolvimento de estudos e indicadores educacionais.[1](https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem)

Conhecer bem os dados dos aplicantes para a prova pode fornecer insights preciosos para a educação brasileira. Para o presente estudo vamos avaliar se dados demográficos, indicadores sociais, de escolaridade dos pais e outras indicações tem correlações com as notas dos candidatos. Muitos insights e respostas podem ser obtidas somente com a análise exploratória de dados.

### **Importando as Bibliotecas e Configurando**

-  Instalando o Pyarrow para converter e salvar arquivo no formato **Parquet**

In [ ]:
# importando as bibliotecas para leitura dos dados e criação de gráficos
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

# configurando pandas para mostrar todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# configurando pandas para não mostrar notação científica para números
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Ignorando alerta
import warnings
warnings.filterwarnings("ignore")

## <a> Carregando os Dados </a>

In [ ]:
# Os dados originais estão em csv, mas salvar em parquet economiza um bocado
df_enem = pd.read_parquet('/content/drive/MyDrive/Erick/enem_2019_amostra.parquet')
df_enem.shape

## <a> Explorando os Dados </a>

- **Salvando apenas** $10$% **da amostra para trabalharmos**

In [ ]:
# Se seu computador não tiver muita memória, vamos trabalhar somente com 10% da base
df_enem = df_enem.sample(n=len(df_enem)//10, random_state=42, replace=False)
df_enem.shape

#### **Salvando um aqruivo Parquet**

In [ ]:
# Salvar o parquet com 10%
df_enem.to_parquet('Enem_2019_amostra_10.parquet')

In [ ]:
# Primeira visualização dos dados (cheirando os dados ;)
df_enem.head()

In [ ]:
# Quantidade de linhas e colunas
df_enem.shape

In [ ]:
# Será que cada linha é mesmo um candidato?
len(df_enem['NU_INSCRICAO'].unique())

In [ ]:
df_enem.dtypes

### **Tratando alguns Dados**

In [ ]:
# Exemplo de list comprehension
lista_colunas_com_nu = [coluna for coluna in df_enem.columns if 'NU' in coluna]
lista_colunas_com_nu


In [ ]:
# Vamos remover as colunas com as respostas, já que não vamos analisar no nível de questões das provas
colunas_questoes = [coluna for coluna in df_enem.columns if coluna.startswith('TX_')]
colunas_questoes

In [ ]:
# Ainda, as colunas do tipo CO_ não servem para nossa análise, são só códigos de coisas que temos siglas (UF, por exemplo)
colunas_codigos = [coluna for coluna in df_enem.columns if coluna.startswith('CO_')]
colunas_codigos

In [ ]:
# Não vamos fazer também uma análise por municípios, então vamos remover tudo que tiver essa string
colunas_municipios = [coluna for coluna in df_enem.columns if 'MUNICIPIO' in coluna]
colunas_municipios

### **Removendo algumas colunas desnecessária**

In [ ]:
# Removendo as colunas (inscrição e ano não são necessários)
colunas_remover = colunas_questoes + colunas_codigos + colunas_municipios + ['NU_INSCRICAO', 'NU_ANO']
df_enem = df_enem.drop(colunas_remover, axis=1)
df_enem.head()

In [ ]:
df_enem.shape

## <a>Dicionário de dados </a>

Percebemos que muitas das colunas possuem somente números, sem descrições. Os dados do ENEM vem acompanhados das descrições com o dicionário, explicando o que cada coluna representa. Seguem algumas que vamos utilizar na análise

Estado Civil (TP_ESTADO_CIVIL)

| Código | Descrição                               |
|--------|-----------------------------------------|
|    0   | Não informado                           |
|    1   | Solteiro(a)                             |
|    2   | Casado(a)/Mora   com companheiro(a)     |
|    3   | Divorciado(a)/Desquitado(a)/Separado(a) |
|    4   | Viúvo(a)                                |

Cor/Raça (TP_COR_RACA)

| Código | Descrição     |
|--------|---------------|
| 0      | Não declarado |
| 1      | Branca        |
| 2      | Preta         |
| 3      | Parda         |
| 4      | Amarela       |
| 5      | Indígena      |

Tipo Nacionalidade

| Código | Descrição                                       |
|--------|-------------------------------------------------|
| 0      | Não informado                                   |
| 1      | Brasileiro(a)                                   |
| 2      | Brasileiro(a)   Naturalizado(a)                 |
| 3      | Estrangeiro(a)                                  |
| 4      | Brasileiro(a)   Nato(a), nascido(a) no exterior |
| 5      | Indígena                                        |

Situação de Conclusão Nível Médio

| Código | Descrição                                              |
|--------|--------------------------------------------------------|
| 1      | Já concluí o   Ensino Médio                            |
| 2      | Estou   cursando e concluirei o Ensino Médio em 2019   |
| 3      | Estou   cursando e concluirei o Ensino Médio após 2019 |
| 4      | Não   concluí e não estou cursando o Ensino Médio      |

Finalmente, as colunas de notas

| Coluna     | Descrição                             |
|------------|---------------------------------------|
| NU_NOTA_CN | Nota da prova de Ciências da Natureza |
| NU_NOTA_CH | Nota da prova de Ciências Humanas     |
| NU_NOTA_LC | Nota da prova de Linguagens e Códigos |
| NU_NOTA_MT | Nota da prova de Matemática           |


## <a> Começando com estatística descritiva </a>

Conhecer bem as medidas estatísticas, de tendência central, dispersão, separatrizes, distribuições, é essencial para conhecermos melhor os dados em que estamos trabalhando. Qual a distribuição por sexo? Raça? Localização geográfica?

In [ ]:
df_enem.describe()

In [ ]:
colunas_notas = [coluna for coluna in df_enem if 'NOTA' in coluna]
colunas_notas

In [ ]:
# Vamos ficar só com a nota principal da redação? Removendo as outras
colunas_notas = [coluna for coluna in colunas_notas if 'COMP' not in coluna]
colunas_notas

### **Boxplot**

In [ ]:
# Analisando as notas
sns.set_style("darkgrid")
figura, graficos = plt.subplots(1, 4, sharey=True, figsize=(8,6))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.boxplot(data=df_enem, y=coluna_1, ax=graficos[i])


### **Distribuição das Notas**

In [ ]:
# Analisando as distribuições das notas
sns.set_style("darkgrid")
figura, graficos = plt.subplots(4, 1, sharex=True, figsize=(8,6))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.histplot(data=df_enem[coluna_1], ax=graficos[i]).set_title(coluna_1)

### **Notas da Redação**

In [ ]:
# Analisando a nota da redação
sns.histplot(data=df_enem['NU_NOTA_REDACAO']).set_title('NU_NOTA_REDACAO');

In [ ]:
# Não ficou legal porque as notas não são muito esparsas, vamos tentar um gráfico de densidade
sns.kdeplot(data=df_enem['NU_NOTA_REDACAO']).set_title('NU_NOTA_REDACAO');

## <a> Analisando as informações demográficas </a>

In [ ]:
df_enem.head()

In [ ]:
# Que índices feios, vamos reiniciar
df_enem = df_enem.reset_index(drop=True)
df_enem.head()

In [ ]:
# Analisando a idade dos candidatos
df_enem['NU_IDADE'].value_counts()

In [ ]:
# Analisando a idade dos candidatos
df_enem['NU_IDADE'].value_counts().sort_index()

In [ ]:
sns.kdeplot(data=df_enem, x='NU_IDADE');

In [ ]:
df_enem['TP_SEXO'].value_counts()

In [ ]:
# Em porcentagem
df_enem['TP_SEXO'].value_counts(normalize=True) * 100

In [ ]:
sns.countplot(data=df_enem, x='TP_SEXO');

- **Informações do Estado Civil**

In [ ]:
# Informações do Estado Civil
'''
0 → Não informado
1 → Solteiro(a)
2 → Casado(a)/Mora com companheiro(a)
3 → Divorciado(a)/Desquitado(a)/Separado(a)
4 → Viúvo(a)
'''
df_enem['TP_ESTADO_CIVIL'].value_counts().sort_index()

In [ ]:
sns.countplot(data=df_enem, x='TP_ESTADO_CIVIL');

- **Informações da Raça**

In [ ]:
'''
0 → Não declarado
1 → Branca
2 → Preta
3 → Parda
4 → Amarela
5 → Indígena
'''
df_enem['TP_COR_RACA'].value_counts().sort_index()

In [ ]:
sns.countplot(data=df_enem, x='TP_COR_RACA');

### **Olhando de onde são as pessoas**

In [ ]:
ufs = df_enem['SG_UF_RESIDENCIA'].value_counts()
ufs

### **Criando um DataFrame com os Estados e Quantidade de Candidatos**

In [ ]:
df_ufs = pd.DataFrame({'uf': ufs.index, 'candidatos': ufs.values})
df_ufs.head()

In [ ]:
import plotly.express as px

fig = px.treemap(data_frame=df_ufs,
    path = ['uf'],
    values = 'candidatos'
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

### Analisando o Questionário Socioeconômico

In [ ]:
# Até que série seu pai, ou o homem responsável por você, estudou?
'''
A → Nunca estudou.
B → Não completou a 4ª série/5º ano do Ensino Fundamental.
C → Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
D → Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
E → Completou o Ensino Médio, mas não completou a Faculdade.
F → Completou a Faculdade, mas não completou a Pós-graduação.
G → Completou a Pós-graduação.
H → Não sei.
'''
df_enem['Q001'].value_counts().sort_index()

### **Analisando o impactos dos que responderam que não sabem**

In [ ]:
# Avaliando o impacto nos dados daqueles que responderam que não sabe
len(df_enem.loc[df_enem['Q001'] == 'H']) / df_enem.shape[0]

In [ ]:
# Até que série sua mãe, ou a mulher responsável por você, estudou?
df_enem['Q002'].value_counts().sort_index()

In [ ]:
# Avaliando o impacto nos dados daqueles que responderam que não sabe
len(df_enem.loc[df_enem['Q002'] == 'H']) / df_enem.shape[0]

### **Filtarndo os pais dos candidatos que não possuem Nível Superior**

In [ ]:
# Criando novas variáveis para delimitar pais que possuem nível superior
df_enem['pai_possui_superior'] = df_enem['Q001'].isin(['F', 'G'])
df_enem['mae_possui_superior'] = df_enem['Q002'].isin(['F', 'G'])
df_enem.head()

In [ ]:
df_enem['pai_possui_superior'].value_counts(normalize=True) * 100

In [ ]:
# Até que série seu pai, ou o homem responsável por você, estudou?
sns.countplot(data=df_enem, x='pai_possui_superior');

In [ ]:
df_enem['mae_possui_superior'].value_counts(normalize=True) * 100

In [ ]:
# Até que série sua mãe, ou a mulher responsável por você, estudou?
sns.countplot(data=df_enem, x='mae_possui_superior');

In [ ]:
lista = ['a', 'b', 'c']
d = dict()

for i, caractere in enumerate(lista):
    print(f'{caractere} na posicao {i}')
    d[caractere] = i

d

### **Analisando a Renda familiar**

In [ ]:
# Analisar a renda familiar. Vamos transformar Q006 em algo numérico?

'''
A → Nenhuma renda.
B → Até R$ 998,00.
C → De R$ 998,01 até R$ 1.497,00.
D → De R$ 1.497,01 até R$ 1.996,00.
E → De R$ 1.996,01 até R$ 2.495,00.
F → De R$ 2.495,01 até R$ 2.994,00.
G → De R$ 2.994,01 até R$ 3.992,00.
H → De R$ 3.992,01 até R$ 4.990,00.
I → De R$ 4.990,01 até R$ 5.988,00.
J → De R$ 5.988,01 até R$ 6.986,00.
K → De R$ 6.986,01 até R$ 7.984,00.
L → De R$ 7.984,01 até R$ 8.982,00.
M → De R$ 8.982,01 até R$ 9.980,00.
N → De R$ 9.980,01 até R$ 11.976,00.
O → De R$ 11.976,01 até R$ 14.970,00.
P → De R$ 14.970,01 até R$ 19.960,00.
Q → Mais de R$ 19.960,00.

'''
from string import ascii_uppercase

dicionario_renda = dict()

# iterando sobre o alfabeto (python não é lindo?)
for i, letra in enumerate(ascii_uppercase):
    if i == 17:
        break
    dicionario_renda[letra] = i

dicionario_renda

In [ ]:
df_enem['Q006'].value_counts()

In [ ]:
df_enem['Q006'].head()

In [ ]:
# função de mapeamento
df_enem['renda_familiar'] = df_enem['Q006'].map(dicionario_renda)

df_enem.head()

In [ ]:
# Análise de renda
sns.countplot(data=df_enem, x='renda_familiar',  palette = 'Set1');

## <a> Relacionar notas com características dos candidatos </a>

Avaliar se as notas dos candidatos é diferente com relação a:
- sexo
- raça
- atendimento especial
- instrucao mae e pai
- renda mensal da familia

### **Criando um DataFrame com as** $Notas$ **e o** $Sexo$

In [ ]:
# Criando df com as notas e sexo dos candidatos
df_sexo = df_enem.loc[:, colunas_notas + ['TP_SEXO']]
df_sexo.head()

In [ ]:
# Analisando as notas por sexo
sns.set_style("darkgrid")
figura, graficos = plt.subplots(1, 4, sharey=True, figsize=(8,6))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.boxplot(data=df_sexo, y=coluna_1, x='TP_SEXO', ax=graficos[i])


![image.png](attachment:image.png)

In [ ]:
# Criando df com as notas e raça dos candidatos
df_raca = df_enem.loc[:, colunas_notas + ['TP_COR_RACA']]
df_raca.head()

In [ ]:
# Analisando as notas por raça
sns.set_style("darkgrid")
figura, graficos = plt.subplots(2, 2, sharey=True, figsize=(10,6))
plt.tight_layout()

for i in range(2):
    coluna_1 = colunas_notas[i*2]
    coluna_2 = colunas_notas[i*2+1]
    sns.boxplot(data=df_raca, y=coluna_1, x='TP_COR_RACA', ax=graficos[i][0])
    sns.boxplot(data=df_raca, y=coluna_2, x='TP_COR_RACA', ax=graficos[i][1])

 - **Pessoas que precisaram de um atendimento personalizado**

In [ ]:
# calcular atendimento especializado
colunas_atendimento_especializado = ['IN_BAIXA_VISAO',
                                     'IN_CEGUEIRA',
                                     'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA',
                                     'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA',
                                     'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO',
                                     'IN_DISLEXIA', 'IN_DISCALCULIA',
                                     'IN_AUTISMO', 'IN_VISAO_MONOCULAR',
                                     'IN_OUTRA_DEF']

df_atendimento_especializado = df_enem.loc[:, colunas_notas]
df_atendimento_especializado['atendimento_especializado'] = df_enem.loc[:, colunas_atendimento_especializado].sum(axis=1) > 0
df_atendimento_especializado.head()

In [ ]:
df_atendimento_especializado['atendimento_especializado'].value_counts()

In [ ]:
df_atendimento_especializado['atendimento_especializado'].value_counts(normalize=True) * 100

- **Analisando as** $Notas$ **por atendimento** $Especializado$

In [ ]:
# Analisando as notas por atendimento especializado
sns.set_style("darkgrid")
figura, graficos = plt.subplots(1, 4, sharey=True, figsize=(8,6))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.boxplot(data=df_atendimento_especializado, y=coluna_1, x='atendimento_especializado', ax=graficos[i])

### **Criando um DataFrame com as** $Notas$ e $Nível$  $de$ $instrução$ dos pais e mães

In [ ]:
# Criando df com as notas e nível de instrução dos pais e mães
df_instrucao_pais = df_enem.loc[:, colunas_notas + ['pai_possui_superior', 'mae_possui_superior']]
df_instrucao_pais.head()

In [ ]:
# Analisando as notas por nivel instrução pais
sns.set_style("darkgrid")
figura, graficos = plt.subplots(1, 4, sharey=True, figsize=(10,4))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.boxplot(data=df_instrucao_pais, y=coluna_1, x='pai_possui_superior', ax=graficos[i])

In [ ]:
# Analisando as notas por nivel instrução mães
sns.set_style("darkgrid")
figura, graficos = plt.subplots(1, 4, sharey=True, figsize=(10,4))
plt.tight_layout()

for i in range(4):
    coluna_1 = colunas_notas[i]
    sns.boxplot(data=df_instrucao_pais, y=coluna_1, x='mae_possui_superior', ax=graficos[i])

### **Analisando a Corelação entre a** $Idade$ **e as** $Notas$

In [ ]:
# Analisando correlação de idades com notas
df_idade = df_enem.loc[:, colunas_notas + ['NU_IDADE']]

sns.scatterplot(data=df_idade, x='NU_IDADE', y='NU_NOTA_CN');

In [ ]:
# Analisando correlação de pearson entre as notas e a idade
correlacoes_idade = df_idade.corr()
sns.heatmap(correlacoes_idade);

In [ ]:
# Analisando a renda
df_renda = df_enem.loc[:, colunas_notas + ['renda_familiar']]

sns.scatterplot(data=df_renda, x='renda_familiar', y='NU_NOTA_CN');


In [ ]:
# Analisando correlação de pearson entre a renda familiar e a idade
correlacoes_renda = df_renda.corr()
sns.heatmap(correlacoes_renda);

## <a> Conclusões </a>

Depois de analisar os dados com o objetivo de avaliar possíveis relações entre características dos candidatos e as notas dos mesmos, podemos afirmar que existe alguma evidência de que há correlações entre as notas:
- raça (principalmente indígenas)
- escolaridade dos pais (notas de matemática muito influenciadas)
- renda familiar na nota de matemática (correlação pouco acima de $0,5$)

Não pudemos confirmar com essa amostra que sexo, idade, atendimento especializado tem relação com as notas.

Ficou claro que a grande maioria dos candidatos é de origem "humilde" com renda familiar inferior a R$ 2.000,00

O candidato $padrão$ do ENEM é mulher, com $18$ anos, solteira, parda, com renda familiar inferior a R$1.500,00 com ambos pai e mãe sem ensino superior completo.
